In [1]:
import gzip
import os

import requests
from rdflib import Graph, DCAT

In [2]:
# TODO: change to a stable version
RIVERBENCH_VERSION = 'dev'
RIVERBENCH_PROFILE = 'flat-triples'
RIVERBENCH_BASE_URL = 'https://w3id.org/riverbench'

BATCH_SIZE = 50_000

In [3]:
r = requests.get(
    f'{RIVERBENCH_BASE_URL}/profiles/{RIVERBENCH_PROFILE}/{RIVERBENCH_VERSION}',
    headers={'Accept': 'text/turtle'}
)

g = Graph()
g.parse(data=r.text, format='turtle')
len(g)

28

In [4]:
datasets = list(g.objects(predicate=DCAT.seriesMember, unique=True))

print('Using the following datasets:')

for dataset in datasets:
    print(' - ' + str(dataset))

Using the following datasets:
 - https://w3id.org/riverbench/datasets/muziekweb/dev
 - https://w3id.org/riverbench/datasets/lod-katrina/dev
 - https://w3id.org/riverbench/datasets/digital-agenda-indicators/dev
 - https://w3id.org/riverbench/datasets/citypulse-traffic/dev
 - https://w3id.org/riverbench/datasets/linked-spending/dev
 - https://w3id.org/riverbench/datasets/dbpedia-live/dev
 - https://w3id.org/riverbench/datasets/assist-iot-weather/dev
 - https://w3id.org/riverbench/datasets/politiquices/dev


/tmp/ipykernel_29210/4168361440.py:1: UserWarning: Code: seriesMember is not defined in namespace DCAT
  datasets = list(g.objects(predicate=DCAT.seriesMember, unique=True))


In [8]:
for dataset in datasets:
    dataset_name = str(dataset.split("/")[-2])
    print(f'Fetching {dataset_name}...')
    
    try:
        if os.stat(f'data/{dataset_name}').st_size > 0:
            print('  Already fetched')
            print()
            continue
    except:
        pass
    
    r = requests.get(
        str(dataset) + '/files/flat_full.nt.gz', 
        stream=True,
    )
    gzip_file = gzip.GzipFile(fileobj=r.raw)
    counter = 0
    os.makedirs(f'data/{dataset_name}', exist_ok=True)
    out_f = None
    
    while True:
        if counter % BATCH_SIZE == 0:
            f_name = f'{counter // BATCH_SIZE:05d}'
            if (counter // BATCH_SIZE) % 50 == 0:
                print(f'  Batch {f_name}')
            
            if out_f:
                out_f.close()
            out_f = open(f'data/{dataset_name}/{f_name}.nt', 'wb')
        line = gzip_file.readline()
        if not line:
            break
        out_f.write(line)
        counter += 1
        
    out_f.close()
    print(f'  Fetched {counter} triples')
    if counter % BATCH_SIZE != 0:
        print('  Removing last batch...')
        os.remove(f'data/{dataset_name}/{counter // BATCH_SIZE:05d}.nt')
    print('  Done')
    print()

Fetching muziekweb...
  Already fetched

Fetching lod-katrina...
  Batch 00000
  Batch 00050


KeyboardInterrupt: 